# Departamento de Engenharia de Teleinformática
# Curso de Graduação em Engenharia de Telecomuniucações
# TI0077 - Inteligência Computacional Aplicada
## Homework 02 - Models for regression - Part 2/2

Parte 1 questões 1 a 3: [Parte 1: Pré-processamento](preprocessing_data_hw02.ipynb)

### Alunos:

- Ezequias Márcio Silva de Santana Júnior - 371887
- Ingrid Simões Moreira Furtado - 375147
- Yuri Sales Ribeiro - 375177

#### Observações:
Para utilizar esse notebook é necessario ter Python3 instalado, juntamente com os pacotes listados abaixo:

- `numpy 1.16.2`
- `pandas 0.24.2`
- `matplotlib 3.0.3`
- `seaborn 0.9.0`
- `scipy 1.2.2` scikit ???

Este notebook, os dados utilizados e eventuais tabelas e figuras salvas encontam-se disponiveis no seguinte 
repositório: https://github.com/ezequiasjunior/ica2019.2

Ao executar este notebook localmente, serão gerados arquivos ```.pdf``` e tabelas ```.xlsx```, ```.tex```.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# scipy ?
# scikit ?
plt.style.use('seaborn')
np.set_printoptions(3)

In [2]:
# Usefull funtions:
def unc_monovar(data):
    #  Display the unconditional mono-variate analysis of a given data
    data_desc = data.describe().T
    data_desc.insert(3, 'skewness', data.skew())
    display(data_desc)

In [3]:
# Loading transformed predictors data:
mos_train = pd.read_excel('hw2-data/Xtrans_mos_train.xlsx')
# Loading outcome data:
mos_Ytrain = pd.read_excel('hw2-data/Y_mos_train.xlsx')
# Info:
unc_monovar(mos_train)
unc_monovar(mos_Ytrain)

,count,mean,std,skewness,min,25%,50%,75%,max
Test Hour,30240.0,-7.102261e-14,1.0,-0.232189,-2.065819,-0.695499,0.122942,0.767480,1.722096
Signal (dBm),30240.0,-1.194020e-12,1.0,0.048122,-3.478094,-0.707686,-0.056239,0.731306,1.632615
Speed (m/s),30240.0,-2.459041e-15,1.0,0.692597,-0.758048,-0.758048,-0.758048,1.169655,1.677909
Distance from site (m),30240.0,1.553501e-14,1.0,-0.021584,-2.426782,-0.688249,0.039530,0.717163,2.819824
Call Test Duration (s),30240.0,7.116715e-13,1.0,-0.000543,-2.188723,-1.000037,0.939352,1.000218,1.000218
Call Test Setup Time (s),30240.0,-6.746335e-14,1.0,1.976446,-2.356230,-0.066317,0.164377,0.418045,27.911710


,count,mean,std,skewness,min,25%,50%,75%,max
MOS,30240.0,2.513519,1.106711,0.265466,1.0,2.1,2.7,2.7,4.4


Use the transformed predictors in the training set to learn an ordinary linear regression model and test the model using the test set (remember to apply the same
pre-processing you used on the training set). Compare the model performance
obtained on the test set with the estimates you would obtain using a resampling
scheme as 5- or 10-fold cross validation: use both the RMSE and R2.

### Modelos:

In [4]:
# Loading the transformed validation dataset:
mos_X = pd.read_excel('hw2-data/Xtrans_mos_valid.xlsx')
# Loading outcome validation data:
mos_Y = pd.read_excel('hw2-data/Y_mos_valid.xlsx')
# Info:
unc_monovar(mos_X)
unc_monovar(mos_Y)

,count,mean,std,skewness,min,25%,50%,75%,max
Test Hour,15121.0,-5.314103e-14,1.0,-0.221734,-2.081963,-0.683479,-0.024196,0.782161,1.731320
Signal (dBm),15121.0,3.120522e-14,1.0,0.045501,-3.494258,-0.713355,-0.058228,0.734563,1.643168
Speed (m/s),15121.0,1.897533e-16,1.0,0.678215,-0.764371,-0.764371,-0.764371,1.174163,1.671729
Distance from site (m),15121.0,2.100047e-14,1.0,-0.021071,-2.414575,-0.681690,0.036183,0.723740,2.844543
Call Test Duration (s),15121.0,-6.235965e-12,1.0,0.007547,-1.067998,-0.996156,-0.996156,1.004140,1.004140
Call Test Setup Time (s),15121.0,-2.318844e-14,1.0,2.321188,-2.331119,-0.040559,0.190482,0.436222,31.288539


,count,mean,std,skewness,min,25%,50%,75%,max
MOS,15121.0,2.504213,1.108045,0.267559,1.0,2.0,2.7,2.7,4.4


In [14]:
from scipy import linalg
def ord_least_squares(xtrain, ytrain):
    #
    b0 = np.ones((xtrain.shape[0], 1))  # column vector of ones
    X = np.concatenate([b0, xtrain], axis=1) # intercept term added
    # Solving via Normal equations
    beta = (linalg.inv(X.T @ X) @ X.T).dot(ytrain)
    return beta

def ridge_regression(xtrain, ytrain, lmbda=1):
    #
    b0 = np.ones((xtrain.shape[0], 1))  # column vector of ones
    X = np.concatenate([b0, xtrain], axis=1) # intercept term added
    # Solving via Normal equations with normF2 constraints:
    Id = np.eye(X.shape[1])
    beta = (linalg.inv((X.T @ X) - lmbda*Id) @ X.T).dot(ytrain)
    return beta

def estimate(beta, xvalid):
    #
    b0 = np.ones((xvalid.shape[0], 1))  # column vector of ones
    X = np.concatenate([b0, xvalid], axis=1) # intercept term added
    y_hat = X.dot(beta)
    return y_hat
    

def perf_score(Y, y_hat):
    #
    mse = ((Y - y_hat)**2).mean()
    r2 =  1 - ((Y - y_hat)**2).sum() / ((Y - Y.mean())**2).sum()
    return np.sqrt(mse), 100*r2 # RMSE and R2(%)

Teste funções:

In [43]:
# fit: ols
ols_coef = ord_least_squares(mos_train.values, mos_Ytrain.values)
y_ols = estimate(ols_coef, mos_X.values)
ols_rmse, ols_r2 = perf_score(mos_Y.values, y_ols)
print(f'OLS metrics w.r.t. validation set:\n coef:\n{ols_coef.T}\n\nRoot Mean Squared Error: {ols_rmse}\nR2:{ols_r2}%')
#fit: ridge
lmbda = 1e-10
ridge_coef = ridge_regression(mos_train.values, mos_Ytrain.values, lmbda)
y_ridge = estimate(ridge_coef, mos_X.values)
ridge_rmse, ridge_r2 = perf_score(mos_Y.values, y_ridge)
print(f'\nRidge (lambda = {lmbda}) metrics w.r.t. validation set:\n coef:\n{ridge_coef.T}\n\nRoot Mean Squared Error: {ridge_rmse}\nR2:{ridge_r2}%')

OLS metrics w.r.t. validation set:
 coef:
[[ 2.514e+00  2.276e-03  2.044e-01  4.413e-02  1.121e-01 -1.593e-01
   4.371e-02]]

Root Mean Squared Error: 1.0860665524846698
R2:3.9213394767109055%

Ridge (lambda = 1e-10) metrics w.r.t. validation set:
 coef:
[[ 2.514e+00  2.276e-03  2.044e-01  4.413e-02  1.121e-01 -1.593e-01
   4.371e-02]]

Root Mean Squared Error: 1.0860665524846698
R2:3.9213394767109055%


In [45]:
ols_coef - ridge_coef

array([[-8.438e-15],
       [ 2.203e-16],
       [-1.110e-15],
       [ 2.776e-17],
       [-8.465e-16],
       [ 1.471e-15],
       [-3.608e-16]])

Teste scikit:

In [7]:
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate

In [49]:
# Models
ols_regr = linear_model.LinearRegression()
# OLS
ols_regr.fit(mos_train.values, mos_Ytrain.values)
ols_Yhat = ols_regr.predict(mos_X.values)
# Metrics
print('OLS Coefficients:\n', ols_regr.intercept_, ols_regr.coef_)
print(f'OLS Root Mean squared error: {np.sqrt(mean_squared_error(mos_Y.values, ols_Yhat))}')
# Explained variance score: 1 is perfect prediction
print(f'OLS Variance score: {r2_score(mos_Y.values, ols_Yhat)}')

# Ridge
ridge_regr = linear_model.Ridge(alpha=1e-5)
ridge_regr.fit(mos_train.values, mos_Ytrain.values)
ridge_Yhat = ridge_regr.predict(mos_X.values)
# Metrics
print('Ridge Coefficients:\n', ridge_regr.intercept_, ridge_regr.coef_)
print(f'Ridge Root Mean squared error: {np.sqrt(mean_squared_error(mos_Y.values, ridge_Yhat))}')
# Explained variance score: 1 is perfect prediction
print(f'Ridge Variance score: {r2_score(mos_Y.values, ridge_Yhat)}')

OLS Coefficients:
 [2.514] [[ 0.002  0.204  0.044  0.112 -0.159  0.044]]
OLS Root Mean squared error: 1.0860665524846698
OLS Variance score: 0.039213394767109055
Ridge Coefficients:
 [2.514] [[ 0.002  0.204  0.044  0.112 -0.159  0.044]]
Ridge Root Mean squared error: 1.0860665524855833
Ridge Variance score: 0.03921339476549268


In [63]:
# CV
cv5o = cross_validate(ols_regr, mos_train.values, mos_Ytrain.values, scoring=['neg_mean_squared_error', 'r2'], cv = 5,return_train_score=True)
cv5r = cross_validate(ols_regr, mos_train.values, mos_Ytrain.values, scoring=['neg_mean_squared_error', 'r2'], cv = 5,return_train_score=True)

cv10o = cross_validate(ols_regr, mos_train.values, mos_Ytrain.values, scoring=['neg_mean_squared_error', 'r2'], cv = 10,return_train_score=True)
cv10r = cross_validate(ols_regr, mos_train.values, mos_Ytrain.values, scoring=['neg_mean_squared_error', 'r2'], cv = 10,return_train_score=True)

In [64]:
print(f'''OLS r2 k5
{np.mean(cv5o['test_r2'])}
{np.mean(cv10o['test_r2'])}

Ridge r2 k5
{np.mean(cv5r['test_r2'])}
{np.mean(cv10r['test_r2'])}
''')

print(f'''OLS rmse k10
{np.mean(np.sqrt(-cv5o['test_neg_mean_squared_error']))}
{np.mean(np.sqrt(-cv10o['test_neg_mean_squared_error']))}

Ridge rmse k10
{np.mean(np.sqrt(-cv5r['test_neg_mean_squared_error']))}
{np.mean(np.sqrt(-cv10r['test_neg_mean_squared_error']))}
''')

OLS r2 k5
0.03634463134143715
0.03629851809929413

Ridge r2 k5
0.03634463134143715
0.03629851809929413

OLS rmse k10
1.0863500652427556
1.086282471619898

Ridge rmse k10
1.0863500652427556
1.086282471619898

